# Lesson 2: First look at MEG and EEG

Neurocampus course "Signals of the whole brain"

Daria Kleeva

dkleeva@gmail.com

February 25, 2025


## Refreshing task before we start

<div style="color: blue;">

Let's implement our first 'proxy' filtering of time series called **moving average**.

- Make a signal that contains: a slow oscillation (e. g., 3 Hz), a fast oscillation (e. g., 40 Hz), some additive noise. Plot the signal for the first 2 seconds. When you look at the raw signal, can you visually tell that there are two oscillations?

- Define a function, which takes an input signal $x$ and some window value $W$ in samples as an input, and the filtered signal $y$ - as an output.

- Rule: for each time index $i$, compute the mean of the last $W$ samples.

- For indices where the window does not fully fit (e. g. the start of the signal), you can either set $y_i$ as Nan, copy the original value or use a smaller window.

- Compare different window sizes. Choose window sizes corresponding roughly to 5 ms, 25 ms, 100 ms etc. Convert ms to samples using sampling rate. 

- For each $W$ compute $y$ using your function, plot $x$ and $y$ on the same figure (zoom into 0-2 seconds), also plot the difference on the different figure. 

- Using peaks detection or upward zero crossings estimate the frequency of the raw $x$ and compare it to the frequencies of each filtered $y$.

- For each $W$, compute the window duration in seconds. Now answer, at which $W$ does the fast oscillating component almost disappear? Does that threshold roughly match the idea that averaging over a window longer than a fraction of a fast period cancels it out? When does the slower component start to suffer?

Bonus:

- Make two separate reference signals (no noise for this part) -- slow and fast with the same frequency content as in the beginning. Define $x_2$ as $slow$ + $0.3 \times fast$. 

- Filter $x_2$ with different $W$. 

- Now compute correlation of filtered signal with the slow and the fast time series. 

- As $W$ increases, what happens to correlation with the fast signal? What happens to correlation with the slow one? Is there a window that kills fast but keeps slow?

- Estimate the delay by finding the shift that maximizes correlation between filtered and original slow component.

</div>

## Typical structures in MNE Python

In [ ]:
import mne
from mne.datasets import sample

In [ ]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = f'{sample_data_folder}/MEG/sample/sample_audvis_raw.fif'

In [ ]:
# edf
#.mat
#.set

In [ ]:
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True)

In [ ]:
raw.info

In [ ]:
raw.first_samp

In [ ]:
raw.info['sfreq']

In [ ]:
raw.first_samp/raw.info['sfreq']

In [ ]:
raw.times

In [ ]:
fig=raw.plot_sensors()

In [ ]:
fig=raw.plot_sensors(ch_type='eeg', show_names=False)

In [ ]:
fig=raw.plot_sensors(ch_type='grad', show_names=True)

In [ ]:
fig=raw.plot_sensors(ch_type='grad', show_names=False, sphere=(0.01, 0.02, 0.01, 0.075))

In [ ]:
fig=raw.plot_sensors(ch_type='grad', show_names=False, sphere=0.1)

## Look at the data

In [ ]:
%matplotlib qt
raw.plot()

In [ ]:
raw_filt = raw.copy().filter(1., 40., fir_design='firwin')

In [ ]:
raw_filt.plot()

In [ ]:
raw.plot_psd(fmin=1., fmax=60., tmax=60., average=False)

In [ ]:
raw_filt.plot_psd(fmin=1., fmax=60., tmax=60., average=False)

## References and montage

In [ ]:
raw_ref = raw.copy().set_eeg_reference(ref_channels=['EEG 001'])
raw_ref.plot()

Phase reversal demo:

In [ ]:
%matplotlib inline
# Time axis
fs = 500
T = 2.0
t = np.arange(int(fs * T)) / fs

# Base background activity
bg = 0.15 * np.sin(2 * np.pi * 8 * t)

# Sharp transient centered at t0 
t0 = 1.0
sigma = 0.015
sharp = np.exp(-0.5 * ((t - t0) / sigma) ** 2)

# Simulate 3 adjacent electrodes: F3 - C3 - P3
# Source maximal at C3, weaker at neighbors
F3 = bg + 0.4 * sharp
C3 = bg + 1.0 * sharp
P3 = bg + 0.4 * sharp

# Bipolar derivations
F3_C3 = F3 - C3
C3_P3 = C3 - P3

fig, ax = plt.subplots(2, 1, figsize=(11, 7), sharex=True)


ax[0].plot(t, F3, label='F3 (referential-like)')
ax[0].plot(t, C3, label='C3 (referential-like)')
ax[0].plot(t, P3, label='P3 (referential-like)')
ax[0].axvline(t0, color='k', ls='--', alpha=0.5)
ax[0].set_title('Sharp transient maximal at C3')
ax[0].set_ylabel('Amplitude')
ax[0].grid(alpha=0.3)
ax[0].legend(frameon=False, ncol=3)


ax[1].plot(t, F3_C3, label='F3 - C3', color='C1')
ax[1].plot(t, C3_P3, label='C3 - P3', color='C2')
ax[1].axvline(t0, color='k', ls='--', alpha=0.5)
ax[1].set_title('Bipolar channels: phase reversal around the sharp transient')
ax[1].set_xlabel('Time (s)')
ax[1].set_ylabel('Amplitude')
ax[1].grid(alpha=0.3)
ax[1].legend(frameon=False)

plt.tight_layout()
plt.show()

In [ ]:
%matplotlib qt

raw_bip = mne.set_bipolar_reference(raw.copy(), anode = ['EEG 001', 'EEG 004', 'EEG 008'], 
                                    cathode = ['EEG 004', 'EEG 008', 'EEG 018'])
raw_bip.plot()

In [ ]:
raw_av = raw.copy().set_eeg_reference(ref_channels='average')
raw_av.plot()

In [ ]:
import mne

# Example (assumes raw already loaded and EEG channels present)
montage = mne.channels.make_standard_montage("standard_1020")

raw.set_montage(montage, match_case=False, on_missing="warn")

print(raw.get_montage())
print(f"EEG channels: {len(mne.pick_types(raw.info, eeg=True))}")

# Optional quick check (spatial layout)
raw.plot_sensors(show_names=True)

In [ ]:
all_montages = mne.channels.get_builtin_montages()
print(f"Total built-in montages: {len(all_montages)}")
for name in all_montages:
    print(name)

In [ ]:
montage = mne.channels.make_standard_montage("standard_1020")

raw.set_montage(montage, match_case=False, on_missing="warn")

print(raw.get_montage())
print(f"EEG channels: {len(mne.pick_types(raw.info, eeg=True))}")


In [ ]:
mapping={'EEG 001': 'Fp1'}
raw.rename_channels(mapping)
raw.copy().pick('eeg').plot_sensors(show_names=True)

## Stimulus channel, events, and annotations

In [ ]:
raw.copy().pick(picks="stim").plot(start=10, duration=10)

In [ ]:
events = mne.find_events(raw, stim_channel="STI 014")

In [ ]:
events

In [ ]:
fig = mne.viz.plot_events(events, raw.info['sfreq'], raw.first_samp)

In [ ]:
testing_data_folder = mne.datasets.testing.data_path()
eeglab_raw_file = testing_data_folder / "EEGLAB" / "test_raw.set"
eeglab_raw = mne.io.read_raw_eeglab(eeglab_raw_file)
print(eeglab_raw.annotations)

In [ ]:
%matplotlib qt
eeglab_raw.plot()

In [ ]:
eeglab_raw.annotations.description

In [ ]:
eeglab_raw.annotations.onset

In [ ]:
eeglab_raw.annotations.duration

In [ ]:
events_from_annot, event_dict = mne.events_from_annotations(eeglab_raw)
print(event_dict)
print(events_from_annot)

In [ ]:
%matplotlib inline
fig = mne.viz.plot_events(events_from_annot, eeglab_raw.info['sfreq'], eeglab_raw.first_samp)

In [ ]:
manual_annot = mne.Annotations(onset=[5, 41], duration=[16, 11], description=["Manual_marker"] * 2)
raw.set_annotations(manual_annot)
(manual_events, manual_event_dict) = mne.events_from_annotations(raw, chunk_duration=1.5)
print(manual_event_dict)
print(manual_events)
fig = mne.viz.plot_events(manual_events, raw.info['sfreq'], raw.first_samp)


In [ ]:
%matplotlib qt
fig = raw.plot()
fig.fake_keypress("a")

## Normal variants of MEG/EEG

In [ ]:
path_to_data = '/Users/dkleeva/Library/CloudStorage/GoogleDrive-dkleeva@gmail.com/My Drive/Teaching/Сигналы целого мозга 2026/Scripts/Data/Normal variants'
epochs = mne.read_epochs(path_to_data + '/Blinks_epochs.fif')


In [ ]:
%matplotlib qt
epochs.plot(group_by='position', n_epochs=1)

In [ ]:
%matplotlib inline
epochs.average().plot_joint()

## Independent component analysis

**Independent Component Analysis (ICA)** is a blind source separation method that assumes observed multichannel signals are linear mixtures of statistically independent latent sources. Formally, we observe $ X = AS $, where $ S $ contains unknown independent components and $ A $ is an unknown mixing matrix; ICA estimates an unmixing matrix $ W \approx A^{-1} $ such that $ \hat{S} = WX $ maximizes statistical independence between components. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import FastICA
from scipy import signal

Let's create independent sources

In [ ]:
rng = np.random.default_rng(0)
sfreq = 250  # Hz
T = 6.0      # seconds
t = np.arange(int(T * sfreq)) / sfreq
n = t.size

s1 = np.sin(2 * np.pi * 8 * t)                                  # sinusoid (8 Hz)
s2 = signal.sawtooth(2 * np.pi * 2 * t, width=0.3)              # non-sinusoidal wave (2 Hz)
s3 = rng.laplace(loc=0.0, scale=1.0, size=n)        


fig, axs = plt.subplots(3, 1, figsize=(10, 6), sharex=True)

axs[0].plot(t, s1, label='Sinusoid (8 Hz)')
axs[0].set_ylabel('Amplitude')
axs[0].legend(loc='upper right')

axs[1].plot(t, s2, label='Sawtooth (2 Hz)', color='orange')
axs[1].set_ylabel('Amplitude')
axs[1].legend(loc='upper right')

axs[2].plot(t, s3, label='Laplace (0 Hz)', color='green')
axs[2].set_ylabel('Amplitude')
axs[2].set_xlabel('Time (s)')
axs[2].legend(loc='upper right')

plt.tight_layout()

Are the sources Gaussian?

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 6), sharex=True)

axs[0].hist(s1, bins=20)
axs[0].set_ylabel('Frequency')
axs[0].set_title('Sinusoid (8 Hz)')

axs[1].hist(s2, bins=20)
axs[1].set_ylabel('Frequency')
axs[1].set_title('Sawtooth (2 Hz)')

axs[2].hist(s3, bins=20)
axs[2].set_ylabel('Frequency')
axs[2].set_title('Laplace (0 Hz)')

plt.tight_layout()


from scipy.stats import kurtosis
print(kurtosis(s1))
print(kurtosis(s2))
print(kurtosis(s3))




It is OK to have at most one Gaussian source for ICA. Then, the decomposition is still identifiable. 

Why:
- Gaussian distributions are rotationally symmetric.
- Any orthogonal rotation of Gaussian variables is still Gaussian.
- If we had more than one Gaussian sources, we could rotate them arbitrarily and ICA would have no unique solution.

In [ ]:
S = np.c_[s1, s2, s3]
S = (S - S.mean(axis=0)) / S.std(axis=0)

Now we mix the sources (in real life we do not know the mixing matrix)

In [ ]:
A = np.array([[1.0,  0.5, 0.2],
              [0.3,  1.0, 0.4],
              [0.2, -0.2, 1.0]])  

X = S @ A.T 

#Add some noise to look realistic
X += 0.02 * rng.standard_normal(size=X.shape)

fig, axs = plt.subplots(3, 1, figsize=(10, 6), sharex=True)

axs[0].plot(t, X[:, 0], label='Mixed signal 1')
axs[0].set_ylabel('Amplitude')
axs[0].legend(loc='upper right')

axs[1].plot(t, X[:, 1], label='Mixed signal 2')
axs[1].set_ylabel('Amplitude')
axs[1].legend(loc='upper right')    

axs[2].plot(t, X[:, 2], label='Mixed signal 3')
axs[2].set_ylabel('Amplitude')
axs[2].set_xlabel('Time (s)')
axs[2].legend(loc='upper right')

plt.tight_layout()



Run ICA on mixtures

In [ ]:
ica = FastICA(n_components=3, whiten="unit-variance", random_state=0, max_iter=2000)
S_hat = ica.fit_transform(X)     
A_hat = ica.mixing_   

In [ ]:
A_hat

In [ ]:
A

Let's look how well we recovered the signal

In [ ]:
fig, axs = plt.subplots(3, 1, figsize=(10, 6), sharex=True)

axs[0].plot(t, S_hat[:, 0], label='Reconstructed source 1')
axs[0].set_ylabel('Amplitude')
axs[0].legend(loc='upper right')

axs[1].plot(t, S_hat[:, 1], label='Reconstructed source 2')
axs[1].set_ylabel('Amplitude')
axs[1].legend(loc='upper right')
plt.plot(t, S_hat[:, 2], label='Reconstructed source 3')
plt.legend()
plt.xlabel('Time (s)')
plt.ylabel('Amplitude')
plt.tight_layout()

Now let's switch to the real data

In [ ]:
import os
from mne.preprocessing import ICA
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(
    sample_data_folder, "MEG", "sample", "sample_audvis_filt-0-40_raw.fif"
)
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.pick('eeg')
raw.load_data()
raw.filter(1., 40., fir_design='firwin')

In [ ]:
ica = ICA(n_components=15, max_iter="auto", random_state=97)
ica.fit(raw)

In [ ]:
explained_var_ratio = ica.get_explained_variance_ratio(raw)
print(f"Fraction of variance explained by all components: {explained_var_ratio}")


In [ ]:
%matplotlib qt
ica.plot_sources(raw)

In [ ]:
%matplotlib inline
ica.plot_components()
plt.show()

In [ ]:
fig = ica.plot_overlay(raw, exclude=[0], picks="eeg")

In [ ]:
ica.plot_properties(raw, picks=range(15))

In [ ]:
ica.exclude = []
clean_raw = raw.copy()
ica.apply(clean_raw)


In [ ]:
%matplotlib qt
clean_raw.plot()

In [ ]:
raw.plot()